
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction which is an angle in degrees and convert it into a point on the unit circle. Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

data = pd.read_csv("data/smhi.csv", delimiter=";", skiprows=10)
data_cleaned = data[["Vindriktning", "Vindhastighet"]]

problem1_wind_direction = data_cleaned["Vindriktning"].to_numpy()
problem1_wind_speed = data_cleaned["Vindhastighet"].to_numpy()

In [14]:
# The wind direction is given as a compass direction in degrees (0-360)
# convert it to x and y coordinates using the standard mathematical convention
problem1_wind_direction_rad = np.radians(problem1_wind_direction)

problem1_wind_direction_x_coordinate = np.cos(problem1_wind_direction_rad)
problem1_wind_direction_y_coordinate = np.sin(problem1_wind_direction_rad)

problem1_wind_velocity_x_coordinate = problem1_wind_speed * problem1_wind_direction_x_coordinate
problem1_wind_velocity_y_coordinate = problem1_wind_speed * problem1_wind_direction_y_coordinate

In [15]:
# Put the average wind velocity x and y coordinates here in these variables
problem1_average_wind_velocity_x_coordinate = np.mean(problem1_wind_velocity_x_coordinate)
problem1_average_wind_velocity_y_coordinate = np.mean(problem1_wind_velocity_y_coordinate)

# First calculate the angle of the average wind velocity vector in degrees
problem1_average_wind_velocity_angle_degrees = np.degrees(np.arctan2(
    problem1_average_wind_velocity_y_coordinate,
    problem1_average_wind_velocity_x_coordinate))

# Then calculate the average angle of the wind direction in degrees (using the wind direction in the data)
problem1_average_wind_direction_angle_degrees = np.mean(problem1_wind_direction)

# Finally, are they the same? Answer as a boolean value (True or False)
problem1_same_angle = np.isclose(
    problem1_average_wind_velocity_angle_degrees,
    problem1_average_wind_direction_angle_degrees,
    atol=1e-5
)

# print("Average Wind Velocity Vector Angle:", problem1_average_wind_velocity_angle_degrees)
# print("Average Wind Direction Angle:", problem1_average_wind_direction_angle_degrees)
# print("Are they the same?", problem1_same_angle)

In [16]:
problem1_wind_velocity = np.stack((problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate), axis=1)

problem1_wind_velocity_covariance_matrix = np.cov(problem1_wind_velocity, rowvar=False)

#print(problem1_wind_velocity_covariance_matrix)

---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [17]:

df_train = pd.read_csv('data/indoor_train.csv')

print(df_train)

      Position X   Position Y  Position Z  Location
0           32.0         15.0         4.4        18
1            8.0         17.0         1.5         9
2            4.0         13.0         4.4        13
3           39.0         16.0         4.4        18
4           34.0         12.0         7.6        15
...          ...          ...         ...       ...
1149        39.0          9.0         1.5        11
1150        38.0         26.0         4.4         6
1151         4.0          7.0         1.5         2
1152        21.0         19.0         1.5        17
1153        25.0         19.0         7.6        19

[1154 rows x 4 columns]


In [18]:

Xtrain = df_train[['Position X', ' Position Y', 'Position Z']].astype('float64')
Ytrain = df_train['Location'].astype('int64')

print(Xtrain.dtypes)

Position X     float64
 Position Y    float64
Position Z     float64
dtype: object


In [19]:

import sklearn as skl
from sklearn.svm import SVC

svc_train = SVC(kernel = 'linear')

model = svc_train.fit(Xtrain, Ytrain)



---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [20]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('data/spam.csv', encoding='latin1',
                   usecols=[0,1])

data['label'] = data.iloc[:, 0].map({'ham': 0, 'spam': 1})
data['contains_free'] = data.iloc[:, 1].str.contains(r'\bfree\b', case=False, na=False).astype(int)
data['contains_prize'] = data.iloc[:, 1].str.contains(r'\bprize\b', case=False, na=False).astype(int)
data['contains_win'] = data.iloc[:, 1].str.contains(r'\bwin\b', case=False, na=False).astype(int)

problem3_X = data[['contains_free', 'contains_prize', 'contains_win']].to_numpy()
problem3_Y = data['label'].to_numpy()

X_train, X_temp, Y_train, Y_temp = train_test_split(problem3_X, problem3_Y, test_size=0.6, random_state=42, stratify=problem3_Y)
X_calib, X_test, Y_calib, Y_test = train_test_split(X_temp, Y_temp, test_size=2/3, random_state=42, stratify=Y_temp)

problem3_X_train = X_train
problem3_X_calib = X_calib
problem3_X_test = X_test

problem3_Y_train = Y_train
problem3_Y_calib = Y_calib
problem3_Y_test = Y_test

print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)

(2228, 3) (1114, 3) (2230, 3) (2228,) (1114,) (2230,)


In [21]:


class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None

    # define the objective/cost/loss function we want to minimise
    def loss(self, X, Y, coeffs):
        G = lambda x: 1 / (np.exp(-x) + 1)
        beta0 = coeffs[0]
        beta1 = coeffs[1:]
        terms = beta0 + beta1@X.T
        P = G(terms)

        loss = (- np.sum(Y * np.log(P) + (1-Y)*np.log(1-P))) / len(X)
        return loss

    def fit(self, X, Y):
        import numpy as np
        from scipy import optimize

        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X, Y, coeffs)
        initial_arguments = np.zeros(shape=X.shape[1] + 1)
        self.result = optimize.minimize(opt_loss, initial_arguments, method='cg')
        self.coeffs = self.result.x

    def predict(self, X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x) / (1 + np.exp(x))
            return np.round(10 * G(
                np.dot(X, self.coeffs[1:]) + self.coeffs[0])) / 10  # This rounding is to help you with the calibration

In [29]:
from sklearn.tree import DecisionTreeRegressor

problem3_ps = ProportionalSpam()
problem3_ps.fit(problem3_X_train, problem3_Y_train)

problem3_X_pred = problem3_ps.predict(problem3_X_calib)
print(problem3_X_pred)

Calibrator = DecisionTreeRegressor()
Calibrator.fit(problem3_X_pred.reshape(-1,1), problem3_Y_calib)

[0.1 0.1 0.1 ... 0.1 0.1 0.1]


DecisionTreeRegressor()

In [30]:

problem3_test_pred = problem3_ps.predict(problem3_X_test)

problem3_final_predictions = Calibrator.predict(problem3_test_pred.reshape(-1,1))

print(problem3_final_predictions)

[0.08812261 0.08812261 0.08812261 ... 0.08812261 0.08812261 0.08812261]


---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [24]:
try:
    import numpy as np

    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1, 0, 1], [0, 1, 1]]), np.array([1, 0]), np.array([1.2, 0.4, 0.3, 0.9]))
    assert (np.abs(test_loss - 1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

Your loss was correct for a test point
